In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())
print(train_df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


(7614, 6)


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-32a081f8d74d>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-32a081f8d74d>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=7)

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/59.8M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        # Log training loss
        if self.state.global_step % self.args.logging_steps == 0:
            print(f"Step {self.state.global_step}, Training Loss: {loss.item()}")

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)
    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         logging_dir="./logs",
#         logging_steps=10,
#         save_steps=500,
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#         logging_first_step=True,
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )
#     print("Starting training...")
#     start_time = time.time()
#     trainer.train()
#     end_time = time.time()
#     training_duration = end_time - start_time
#     print(f"Training completed in {training_duration:.2f} seconds.")

#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [11]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("<<=================Best trial=========================>>")
# print(study.best_trial)
# print("<<==========================================>>")

In [12]:
# best_hyperparams = study.best_trial.params
training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=15,  # Increase epochs
    per_device_train_batch_size=16,  # Increase batch size
    per_device_eval_batch_size=32 ,  # Double batch size for evaluation
    learning_rate=3e-5,  # Slightly larger learning rate
    weight_decay=0.01,  # Higher weight decay for regularization
    warmup_steps=500,  # Increased warmup steps
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,  # Adjust based on hardware
    report_to="none",
)



trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

print("Starting training...")
start_time = time.time()

trainer.train()

end_time = time.time()
training_duration = end_time - start_time
print("=====================Training Time=====================")
print(f"Training completed in {training_duration:.2f} seconds.")
print("=======================================================")

Starting training...


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2850


Step 0, Training Loss: 0.6951097846031189
Step 0, Training Loss: 0.6945341229438782


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.513120,0.000000,1.000000,0.000000,0.000000
1,No log,0.341502,0.000000,1.000000,0.000000,0.000000
2,0.478300,0.306374,0.428102,0.894588,0.422806,0.400358
3,0.478300,0.243708,0.665135,0.891526,0.657912,0.637018
4,0.478300,0.205932,0.732764,0.878103,0.725176,0.707312
5,0.256900,0.179266,0.787919,0.896922,0.781550,0.760983
6,0.256900,0.167536,0.783322,0.899378,0.777066,0.758243
7,0.165300,0.153908,0.821405,0.909498,0.813581,0.800062
8,0.165300,0.146907,0.803020,0.931133,0.794363,0.801775
9,0.165300,0.143001,0.819435,0.919645,0.812940,0.805203


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin


...

Step 2000, Training Loss: 0.07398851215839386
Step 2000, Training Loss: 0.0608024038374424


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-2500
Configuration saved in ./best_results/checkpoint-2500/config.json
Model weights saved in ./best_results/checkpoint-2500/pytorch_model.bin


Step 2500, Training Loss: 0.09614865481853485
Step 2500, Training Loss: 0.11311937123537064


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




=====================Training Time=====================
Training completed in 461.35 seconds.


In [13]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


### Model Saving

In [14]:
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_java'
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_huawei-noah_TinyBERT_General_4L_312D_java')
tokenizer.save_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java')
tokenizer = AutoTokenizer.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer')

Saving model checkpoint to ./best_model_huawei-noah_TinyBERT_General_4L_312D_java
Configuration saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model weights saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/pytorch_model.bin
tokenizer config file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_huawei-noah_TinyBERT_General_4L_312D_java",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
   

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) ... array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])]


In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("=================Evaluation Metrics================")
print("Evaluation Metrics:", metrics)
print("===================================================")

print("=================Inference Time================")
print(f"Inference Time: {inference_time:.2f} seconds")
print("===================================================")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 32


=================Evaluation Metrics================
Evaluation Metrics: {'accuracy': 0.7571014492753623, 'precision': 0.8114856482018756, 'recall': 0.7564766839378239, 'f1': 0.7575978110120086}
=================Inference Time================
Inference Time: 3.03 seconds


In [18]:
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_java'
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/training_args.bin (deflated 51%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json (deflated 54%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/pytorch_model.bin (deflated 7%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer.json (deflated 71%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json (deflated 41%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/vocab.txt (deflated 53%)


In [19]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip "/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/"
!cp best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip "/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/"

Mounted at /content/drive
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/': Not a directory


### Load and Test Model

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip' -d './'

unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip, /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip.zip or /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip, /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip.zip or /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip.ZIP.


In [22]:
best_model_huawei_noah_TinyBERT_General_4L_312D_java = 'best_model_huawei-noah_TinyBERT_General_4L_312D_java'
best_model_huawei_noah_TinyBERT_General_4L_312D_java_tokenizer = 'best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

In [23]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_java)
tokenizer = AutoTokenizer.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_java_tokenizer)

loading configuration file best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model config BertConfig {
  "_name_or_path": "best_model_huawei-noah_TinyBERT_General_4L_312D_java",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
 

In [24]:
text = "equals the @code match field. | AbstractContra...."
encoded_input = tokenizer(text, return_tensors='pt')
start_time = time.time()
output = model(**encoded_input)
end_time = time.time()
prediction_time = end_time - start_time
output

print("=============Prediction Time==============")
print(prediction_time)
print("==========================================")

=============Prediction Time==============
0.12822675704956055


In [25]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 1
